In [1]:
import flickrapi
import urllib.request
import numpy as np
import skimage.io
import skimage.transform
import os
import shutil
import warnings
from IPython.display import clear_output

In [2]:
api_key = u'52c68de05a004c919bebf5cf337e8814'
api_secret = u'a3a39bf39168ba7c'
flickr = flickrapi.FlickrAPI(api_key, api_secret)

In [3]:
def resize_im(im0,nrow,ncol):
    nrow0 = im0.shape[0]
    ncol0 = im0.shape[1]
    nchan = im0.shape[2]
    if (ncol0 > nrow0):
        pad = (ncol0-nrow0)//2
        im = np.zeros((ncol0,ncol0,nchan),dtype=np.uint8)
        im[pad:pad+nrow0,:,:] = im0
    elif (nrow0 >= ncol0):
        pad = (nrow0-ncol0)//2
        im = np.zeros((nrow0,nrow0,nchan),dtype=np.uint8)        
        im[:,pad:pad+ncol0,:] = im0
    im = skimage.transform.resize(im,(nrow,ncol),mode='constant')
    return im

In [4]:
def load_images(keywords):
    if os.path.isdir('train'):
        shutil.rmtree('train')
    os.mkdir('train')
    
    if os.path.isdir('test'):
        shutil.rmtree('test')
    os.mkdir('test')
    
    for keyword in keywords:
        
        if os.path.isdir('train/' + keyword):
            shutil.rmtree('train/' + keyword)
        os.mkdir('train/' + keyword)
        
        if os.path.isdir('test/' + keyword):
            shutil.rmtree('test/' + keyword)
        os.mkdir('test/' + keyword)
        
        photos = flickr.walk(text=keyword,
                         tag_mode='all',
                         tags= keyword, 
                         extras='url_c', 
                         sort='relevance',
                         per_page=2000)
        
        full_size_fn = 'full_size'
        train_images = 1000
        test_images = 300
        nimage = 1300
        i = 0
        nrow = 150
        ncol = 150
        
        for photo in photos:
            url = photo.get('url_c')
            
            if not (url is None):
                try:
                    urllib.request.urlretrieve(url, full_size_fn)
                except urllib.error.HTTPError:
                    continue
                    
                im = skimage.io.imread(full_size_fn)
                
                if not (len(im.shape) is 3):
                    continue
                    
                im1 = resize_im(im, nrow, ncol)
                
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    im2 = skimage.img_as_ubyte(im1)
                    
                folder = 'train/' if i < train_images else 'test/'
                index = i if i < train_images else i - train_images
                local_name = '{0:s}/{1:s}_{2:04d}.jpg'.format(folder + keyword, keyword, index)
                skimage.io.imsave(local_name, im2) 
                clear_output()
                print(local_name)
                i += 1 
                
            if (i >= nimage):
                os.remove(full_size_fn)
                clear_output()
                print('Training and testing images loaded')
                break

In [5]:
keywords = ['bicycle', 'car']
load_images(keywords)

Training and testing images loaded
